In [2]:
import tensorflow as tf
import keras
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize values

x_train_processed = x_train/255
x_test_processed = x_test/255

# Add channel dimension

x_train_processed = x_train_processed[:,:,:,np.newaxis]
x_test_processed = x_test_processed[:,:,:,np.newaxis]

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
print("Training size: {}".format(len(x_train)))
print("Test size: {}".format(len(x_test)))

Training size: 60000
Test size: 10000


In [5]:
# check dataset balance
print(np.bincount(y_train))
print(np.bincount(y_test))

[5923 6742 5958 6131 5842 5421 5918 6265 5851 5949]
[ 980 1135 1032 1010  982  892  958 1028  974 1009]


In [ ]:
# Show examples in dataset

def array_to_rgb(arr, denormalize=1):
    shape=arr.shape + (3,)
    img=np.zeros(shape=shape)
    for i in range(3):
        img[:,:,i]=arr*denormalize
    return img

r,c=2,6
index=np.random.randint(0,len(x_train)-r*c)
fig = make_subplots(rows=r, cols=c, subplot_titles=["Label: " + str(y_train[i]) for i in range(index,index+r*c)])
for i in range(r):
    for j in range(c):
        fig.add_trace(
            go.Image(name=str(y_train[index]), z=array_to_rgb(x_train[index])), 
            i+1,j+1)
        index+=1
fig.show()

In [ ]:
# write one-hot encoder
def one_hot_encoder(int_array):
    one_hot=np.zeros((len(int_array),(len(set(int_array)))))
    one_hot[np.arange(int_array.size),int_array]=1
    return one_hot


In [ ]:
# one-hot encode
y_train_one_hot=one_hot_encoder(y_train)
y_test_one_hot=one_hot_encoder(y_test)

start=np.random.randint(0,len(y_train-10))
end=start+10

print(y_train[start:end])
print(y_train_one_hot[start:end])

[2 3 3 5 2 3 3 4 5 8]
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [ ]:
# set-up and train CNN

model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(x_train_processed[0].shape)),
                                    tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Conv2D(16,(3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D((2,2)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(256, activation='relu'),
                                    tf.keras.layers.Dense(10, activation='softmax')
                                    ])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

history = model.fit(x=x_train_processed,y=y_train_one_hot,epochs=20,validation_data=(x_test_processed,y_test_one_hot))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dropout (Dropout)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               1

In [ ]:
# Save model as Savedmodel
model.save('MNIST_model')

# Convert the model to tflite model
converter = tf.lite.TFLiteConverter.from_saved_model('/content/MNIST_model')
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)